In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("data/Train.csv")
df.head()

In [ ]:
df.nunique()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['STA'].replace('\.',':', regex=True,inplace=True)

In [ ]:
df['STD'] = pd.to_datetime(df['STD'])
df['STA'] = pd.to_datetime(df['STA'])
df['DATOP'] = pd.to_datetime(df['DATOP'])

In [ ]:
df['time_diff'] = (df['STA']-df['STD']).dt.total_seconds()/60

In [ ]:
df